# Importing some packages

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

# Model training

Creating higher-order function for a training step

In [5]:
def make_train_step_fn(model, optimizer, lossfn):

    def perform_train_step(x, y):

        model.train()

        y_hat = model(x)

        loss = lossfn(y_hat, y)

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        return loss.item()
    
    return perform_train_step